In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Step 1: Data Preparation
train_dir = 'Desktop/ANN/data/training_images'
validation_dir = 'Desktop/ANN/data/testing_images'
image_size = (224, 224)
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
# Step 2: Model Architecture
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze ResNet50 layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Step 3: Training
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator,
                    steps_per_epoch=len(train_generator),
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=len(validation_generator))

In [ ]:
# Step 6: Comparative Analysis
# Define different optimization algorithms
optimizers = {
    "Adam": Adam(),
    "SGD": SGD()
}

for optimizer_name, optimizer in optimizers.items():
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(train_generator,
                        steps_per_epoch=len(train_generator),
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=len(validation_generator))
    val_pred = model.predict(validation_generator, steps=validation_steps)
    val_pred_binary = np.round(val_pred)
    accuracy = accuracy_score(val_true_labels, val_pred_binary)
    print("Validation Accuracy with", optimizer_name, ":", accuracy)
